In [2]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Revise
using EasyHybrid
using Lux
using Optimisers
using WGLMakie
using Random
using LuxCore
using CSV, DataFrames
using EasyHybrid.MLUtils
using Statistics
using Plots
using JLD2

  Activating project at `/mnt/tupi/HybridModeling/EasyDensity.jl`
    Updating registry at `/opt/julia/registries/General.toml`
   Installed Unitful ─── v1.26.0
   Installed StatsBase ─ v0.34.9
    Updating `/mnt/tupi/HybridModeling/EasyDensity.jl/Project.toml`
⌃ [cbdf2221] + AlgebraOfGraphics v0.11.10
  [94b1ba4f] + AxisKeys v0.2.17
  [336ed68f] + CSV v0.10.15
  [13f3f980] + CairoMakie v0.15.8
  [a93c6f00] + DataFrames v1.8.1
  [61bb816a] + EasyHybrid v0.1.7
  [587475ba] + Flux v0.16.5
  [e9467ef8] + GLMakie v0.13.8
  [033835bb] + JLD2 v0.6.3
  [b2108857] + Lux v1.27.0
  [bb33d45b] + LuxCore v1.4.2
  [872c559c] + NNlib v0.9.31
  [67456a42] + OhMyThreads v0.8.3
  [3bd65402] + Optimisers v0.4.6
  [91a5bcdd] + Plots v1.41.2
  [295af30f] + Revise v3.12.3
  [2913bbd2] + StatsBase v0.34.9
  [f3b207a7] + StatsPlots v0.15.8
  [0c5d862f] + Symbolics v7.1.1
  [276b4fcb] + WGLMakie v0.13.8
  [e88e6eb3] + Zygote v0.7.10
    Updating `/mnt/tupi/HybridModeling/EasyDensity.jl/Manifest.toml`
  [47edc

In [3]:
# 03 - flexiable BD, both oBD and mBD will be learnt by NN
testid = "03_hybridNN";
version = "v20251125"
results_dir = joinpath(@__DIR__, "eval");
target_names = [:BD, :SOCconc, :CF, :SOCdensity];

# input
df = CSV.read(joinpath(@__DIR__, "data/lucas_preprocessed_$version.csv"), DataFrame; normalizenames=true)
println(size(df))

# scales
scalers = Dict(
    :SOCconc   => 0.151, # g/kg, log(x+1)*0.151
    :CF        => 0.263, # percent, log(x+1)*0.263
    :BD        => 0.529, # g/cm3, x*0.529
    :SOCdensity => 0.167, # kg/m3, log(x)*0.167
);

# mechanistic model
function SOCD_model(; SOCconc, CF, oBD, mBD)
    ϵ = 1e-7

    # invert transforms
    soct = (exp.(SOCconc ./ scalers[:SOCconc]) .- 1) ./ 1000
    soct = clamp.(soct, ϵ, Inf)

    cft = (exp.(CF ./ scalers[:CF]) .- 1) ./ 100
    cft = clamp.(cft, 0, 0.99)

    # compute BD safely
    som = 1.724f0 .* soct
    denom = som .* mBD .+ (1f0 .- som) .* oBD
    denom = clamp.(denom, ϵ, Inf)

    BD = (oBD .* mBD) ./ denom
    BD = clamp.(BD, ϵ, Inf)

    # SOCdensity
    SOCdensity = soct .* 1000 .* BD .* (1 .- cft)
    SOCdensity = clamp.(SOCdensity, ϵ, Inf)

    # scale
    SOCdensity = log.(SOCdensity) .* scalers[:SOCdensity]
    BD = BD .* scalers[:BD]

    return (; BD, SOCconc, CF, SOCdensity, oBD, mBD)
end


# param bounds
parameters = (
    SOCconc = (0.01f0, 0.0f0, 1.0f0),   # fraction
    CF      = (0.15f0, 0.0f0, 1.0f0),   # fraction,
    oBD     = (0.20f0, 0.05f0, 0.40f0),  # also NN learnt, g/cm3
    mBD     = (1.20f0, 0.75f0, 2.0f0),  # NN leanrt
)

# define param for hybrid model
neural_param_names = [:SOCconc, :CF, :mBD, :oBD]
# global_param_names = [:oBD]
forcing = Symbol[]
targets = [:BD, :SOCconc, :SOCdensity, :CF]       # SOCconc is both a param and a target

# predictor
predictors = Symbol.(names(df))[18:end-6]; # CHECK EVERY TIME 
nf = length(predictors)




(56117, 385)


In [5]:
# hyperparameters
# search space
hidden_configs = [ 
    (512, 256, 128, 64, 32, 16),
    (512, 256, 128, 64, 32), 
    (256, 128, 64, 32, 16),
    (256, 128, 64, 32),
    (256, 128, 64),
    (128, 64, 32, 16),
    (128, 64, 32),
    (128, 64),
    (64, 32, 16)
];
batch_sizes = [128, 256, 512];
lrs = [1e-3, 5e-4, 1e-4];
activations = [relu, tanh, swish, gelu];

configs = [(h=h, bs=bs, lr=lr, act=act)
           for h in hidden_configs
           for bs in batch_sizes
           for lr in lrs
           for act in activations]

println(length(configs))


324


In [4]:
# cross-validation
k = 5;
folds = make_folds(df, k = k, shuffle = true);
rlt_list_param = Vector{DataFrame}(undef, k)
rlt_list_pred = Vector{DataFrame}(undef, k)  

@info "Threads available: $(Threads.nthreads())"

[ Info: Threads available: 96


In [ ]:

@time for test_fold in 1:k
    @info "Training outer fold $test_fold of $k"

    train_folds = setdiff(1:k, test_fold)
    train_idx = findall(in(train_folds), folds)
    train_df = df[train_idx, :]
    test_idx  = findall(==(test_fold), folds)
    test_df = df[test_idx, :]

    # track best config for this outer fold
    best_val_loss = Inf
    best_config = nothing
    best_result = nothing
    best_hm = nothing
    results_param = DataFrame(h=String[], bs=Int[], lr=Float64[], act=String[], r2=Float64[], mse=Float64[], best_epoch=Int[], test_fold=Int[])

    Threads.@threads for i in 1:length(configs)
        cfg = configs[i]
    
        h  = cfg.h
        bs = cfg.bs
        lr = cfg.lr
        act = cfg.act
        println("Testing h=$h, bs=$bs, lr=$lr, activation=$act")

        hm_local = constructHybridModel(
            predictors,
            forcing,
            targets,
            SOCD_model,
            parameters,
            neural_param_names,
            [];
            hidden_layers = collect(h),
            activation = act,
            scale_nn_outputs = true,
            input_batchnorm = true,
            start_from_default = true
        )

        rlt = train(
            hm_local, train_df, ();
            nepochs = 200,
            batchsize = bs,
            opt = AdamW(lr),
            training_loss = :mse,
            loss_types = [:mse, :r2],
            shuffleobs = true,
            file_name = "history_$(testid)_fold$(test_fold).jld2",
            random_seed = 42,
            patience = 15,
            yscale = identity,
            monitor_names = [:oBD, :mBD],
            agg = mean,
            return_model = :best,
            show_progress = true,
            plotting = false,
            hybrid_name = "$(testid)_fold$(test_fold)" 
        )

        lock(lk)
        if rlt.best_loss < best_val_loss
            best_val_loss = rlt.best_loss
            best_config = cfg
            best_result = rlt
            best_hm = deepcopy(hm_local)
        end
        unlock(lk)

    end

    # register best hyper paramets
    agg_name = Symbol("mean")
    r2s  = map(vh -> getproperty(vh, agg_name), best_result.val_history.r2)
    mses = map(vh -> getproperty(vh, agg_name), best_result.val_history.mse)
    best_epoch = best_result.best_epoch

    local_results_param = DataFrame(
        h = string(best_config.h),
        bs = best_config.bs,
        lr = best_config.lr,
        act = string(best_config.act),
        r2 = r2s[best_epoch],
        mse = mses[best_epoch],
        best_epoch = best_epoch,
        test_fold = test_fold
    )
    rlt_list_param[test_fold] = local_results_param
    
    (x_test,  y_test)  = prepare_data(best_hm, test_df)
    ps, st = best_result.ps, best_result.st
    
    save(joinpath(results_dir, "$(testid)_bestm_fold$(test_fold)_$(version).jld2"),
         "hm" => best_hm,
         "rlt" => best_result,
         "ps" => ps,
         "st" => st)
    
    ŷ_test, st_test = best_hm(x_test, ps, LuxCore.testmode(st))
    println(propertynames(ŷ_test))
    println(propertynames(ŷ_test.parameters))

    for var in [:BD, :SOCconc, :CF, :SOCdensity, :oBD, :mBD]
        if hasproperty(ŷ_test, var)
            val = getproperty(ŷ_test, var)

            if val isa AbstractVector && length(val) == nrow(test_df)
                test_df[!, Symbol("pred_", var)] = val # per row

            elseif (val isa Number) || (val isa AbstractVector && length(val) == 1)
                test_df[!, Symbol("pred_", var)] = fill(Float32(val isa AbstractVector ? first(val) : val), nrow(test_df))
            end


        end
    end
    
    rlt_list_pred[test_fold] = test_df

end

rlt_param = vcat(rlt_list_param...)
rlt_pred = vcat(rlt_list_pred...)

CSV.write(joinpath(results_dir, "$(testid)_cv.pred_$version.csv"), rlt_pred)
CSV.write(joinpath(results_dir, "$(testid)_hyperparams_$version.csv"), rlt_param)


[ Info: Training outer fold 1 of 5


Testing h=(512, 256, 128, 64, 32), bs=128, lr=0.0005, activation=relu
Testing h=(256, 128, 64, 32, 16), bs=256, lr=0.0005, activation=relu
Testing h=(256, 128, 64, 32, 16), bs=512, lr=0.0001, activation=relu
Testing h=(128, 64, 32, 16), bs=128, lr=0.0005, activation=swish
Testing h=(128, 64, 32), bs=512, lr=0.0001, activation=tanh
Testing h=(64, 32, 16), bs=128, lr=0.001, activation=relu
Testing h=(128, 64, 32), bs=256, lr=0.001, activation=gelu_tanh
Testing h=(256, 128, 64), bs=512, lr=0.001, activation=gelu_tanh
Testing h=(64, 32, 16), bs=256, lr=0.0001, activation=tanh
Testing h=(128, 64, 32), bs=128, lr=0.001, activation=gelu_tanh
Testing h=(256, 128, 64, 32, 16), bs=128, lr=0.0001, activation=relu
Testing h=(64, 32, 16), bs=128, lr=0.001, activation=gelu_tanh
Testing h=(256, 128, 64), bs=256, lr=0.001, activation=relu
Testing h=(128, 64), bs=128, lr=0.0001, activation=tanh
Testing h=(128, 64, 32, 16), bs=128, lr=0.001, activation=gelu_tanh
Testing h=(256, 128, 64), bs=512, lr=0.00